In [ ]:
import kagglehub
manjilkarki_deepfake_and_real_images_path = kagglehub.dataset_download('manjilkarki/deepfake-and-real-images')
# birdy654_cifake_real_and_ai_generated_synthetic_images_path = kagglehub.dataset_download('birdy654/cifake-real-and-ai-generated-synthetic-images')

print('Data source import complete.')

Data source import complete.


In [2]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import logging
import sys

In [4]:
import os

# Limit TensorFlow GPU memory growth
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logs (optional)

import tensorflow as tf

# Check GPU availability
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable dynamic memory growth to avoid grabbing all memory
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"TensorFlow is set to use GPU: {[gpu.name for gpu in gpus]}")
    except RuntimeError as e:
        print(f"ERROR: Could not set memory growth: {e}")
else:
    print("WARNING: No GPU detected. Running on CPU.")

TensorFlow is set to use GPU: ['/physical_device:GPU:0']


In [8]:
import tensorflow as tf
import numpy as np
import os
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
import sys

# ------------- Feature extraction functions ----------------

def extract_lbp_features(gray_image, radius=1, n_points=8):
    from skimage.feature import local_binary_pattern
    try:
        lbp = local_binary_pattern(gray_image, n_points, radius, method='uniform')
        n_bins = int(lbp.max() + 1)
        hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
        if len(hist) == 0:
            expected_bins = n_points + 2
            return np.zeros(expected_bins)
        return hist
    except Exception as e:
        print(f"ERROR: Error processing LBP: {e}")
        return np.zeros(n_points + 2)

def extract_og_features(gray_image, n_bins=8):
    try:
        gx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
        gy = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
        magnitude, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
        hist, _ = np.histogram(angle.ravel(), bins=n_bins, range=(0, 360), weights=magnitude.ravel(), density=False)
        hist_sum = np.sum(hist)
        if hist_sum > 0:
            hist = hist / hist_sum
        else:
            hist = np.zeros(n_bins)
        return hist
    except Exception as e:
        print(f"ERROR: Error in OG extraction: {e}")
        return np.zeros(n_bins)

def extract_ssee_features(image, gray_image, quality=75):
    try:
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
        result, encimg = cv2.imencode('.jpg', image, encode_param)
        if not result:
            return np.array([0.0, 0.0])
        decimg_gray = cv2.imdecode(encimg, cv2.IMREAD_GRAYSCALE)
        if decimg_gray is None:
            return np.array([0.0, 0.0])
        if decimg_gray.shape != gray_image.shape:
            decimg_gray = cv2.resize(decimg_gray, (gray_image.shape[1], gray_image.shape[0]))
        diff = cv2.absdiff(gray_image, decimg_gray)
        mean_diff = np.mean(diff)
        std_diff = np.std(diff)
        return np.array([mean_diff, std_diff])
    except Exception as e:
        print(f"ERROR: Error in SSEE (JPEG Error) extraction: {e}")
        return np.array([0.0, 0.0])

def extract_combined_features_and_image(image_path, target_size=(224, 224)):
    try:
        image = cv2.imread(image_path)
        if image is None:
            return None, None
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        lbp_hist = extract_lbp_features(gray_image)
        og_hist = extract_og_features(gray_image)
        ssee_feats = extract_ssee_features(image, gray_image)

        combined_features = np.concatenate((lbp_hist, og_hist, ssee_feats)).astype(np.float32)

        img_resized = cv2.resize(image, target_size)
        img_preprocessed = resnet_preprocess(np.expand_dims(img_resized, axis=0))[0]

        return img_preprocessed, combined_features
    except Exception as e:
        print(f"ERROR processing {image_path}: {e}")
        return None, None

# ------------- Dataset Preparation ----------------

def list_image_paths_and_labels(base_dir):
    categories = ['Fake', 'Real']
    paths = []
    labels = []
    for label, category in enumerate(categories):
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            continue
        for filename in os.listdir(category_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                paths.append(os.path.join(category_dir, filename))
                labels.append(label)
    return paths, labels

def create_dataset(image_paths, labels, batch_size=32, shuffle=True):
    AUTOTUNE = tf.data.AUTOTUNE

    def _load_function(image_path, label):
        img_path = image_path.numpy().decode()
        img_array, feature_array = extract_combined_features_and_image(img_path)
        if img_array is None or feature_array is None:
            img_array = np.zeros((224, 224, 3), dtype=np.float32)
            feature_array = np.zeros(8 + 10 + 2, dtype=np.float32) # lbp bins + og bins + ssee
        return img_array, feature_array, label

    def _tf_wrapper(image_path, label):
        img, features, lbl = tf.py_function(func=_load_function, inp=[image_path, label],
                                             Tout=[tf.float32, tf.float32, tf.int32])
        img.set_shape((224, 224, 3))
        features.set_shape((20,))
        lbl.set_shape(())
        return (img, features), tf.one_hot(lbl, depth=2)

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.map(_tf_wrapper, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(AUTOTUNE)
    return dataset

# ------------- Usage ----------------

# Paths
base_dir = '/kaggle/input/deepfake-and-real-images/Dataset/'
train_dir = os.path.join(base_dir, 'Train')
test_dir = os.path.join(base_dir, 'Test')

# Prepare file paths and labels
train_paths, train_labels = list_image_paths_and_labels(train_dir)
test_paths, test_labels = list_image_paths_and_labels(test_dir)
train_paths = train_paths[:int(len(train_paths)*0.1)]
train_labels = train_labels[:int(len(train_labels)*0.1)]
test_paths = test_paths[:int(len(test_paths)*0.1)]
test_labels = test_labels[:int(len(test_labels)*0.1)]
# Create datasets
batch_size = 256
train_dataset = create_dataset(train_paths, train_labels, batch_size=batch_size, shuffle=True)
test_dataset = create_dataset(test_paths, test_labels, batch_size=batch_size, shuffle=False)

print("Datasets ready for training and testing!")

Datasets ready for training and testing!


In [9]:
import os
import numpy as np
import tensorflow as tf
import cv2
import logging
import sys
from datetime import datetime

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# Assuming IMG_SIZE and FEATURE_LENGTH are already set properly
IMG_SIZE = 224
NUM_CLASSES = 2


FEATURE_LENGTH = 20

print("Building multi-input ResNet + Features model...")

# --- Image Input Branch (ResNet) ---
image_input = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='image_input')
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)
base_model.trainable = False  # Freeze ResNet layers
image_features = base_model.output
image_features = GlobalAveragePooling2D(name='image_gap')(image_features)

# --- Handcrafted Features Input Branch ---
feature_input = Input(shape=(FEATURE_LENGTH,), name='feature_input')
feature_dense = Dense(64, activation='relu', name='feature_dense_1')(feature_input)
feature_dense = Dense(32, activation='relu', name='feature_dense_2')(feature_dense)

# --- Combine Branches ---
combined = concatenate([image_features, feature_dense], name='concatenate_features')

# --- Classification Head ---
final_dense = Dense(512, activation='relu', name='final_dense_1')(combined)
predictions = Dense(NUM_CLASSES, activation='softmax', name='output_predictions')(final_dense)

# --- Create Model ---
multi_input_model = Model(inputs=[image_input, feature_input], outputs=predictions)

# --- Compile Model ---
multi_input_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Multi-input model built and compiled.")
multi_input_model.summary(print_fn=lambda x: print(x))

# --- Callbacks ---

# Model Checkpoint
checkpoint_path = 'best_model.keras'
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# TensorBoard Callback
log_dir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True
)

# --- Train Model ---
print("Training multi-input model...")

history = multi_input_model.fit(
    train_dataset,  # Make sure train_dataset is prepared as a tuple (X_train_keras, y_train_cat) and batched
    epochs=25,
    validation_data=test_dataset,  # Same as above
    callbacks=[checkpoint_callback, tensorboard_callback],
    verbose=1
)

print("Multi-input model training complete.")
print(f"Best model saved to: {checkpoint_path}")
print(f"TensorBoard logs saved to: {log_dir}")

Building multi-input ResNet + Features model...
Multi-input model built and compiled.


Model: "functional_3"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ image_input[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256

KeyboardInterrupt: 

In [ ]:
model_save_path = 'model.keras'

# Save the entire model (architecture + weights + optimizer state)
multi_input_model.save(model_save_path)

print(f"Model saved successfully to {model_save_path}")